In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
import numpy as np
import seaborn as sns
%matplotlib inline

In [ ]:
def to_binary(signal):
    """
    convert normalized Red1 signal to
    a binary value
    
    signal: float
    """
    if signal > 0: return 1
    else: return 0

In [ ]:
def mean_signal_per_window(a, w=1000):
    """
    mean signal in non-overlapping windows 
    of specified size
    
    a: list of floats
    """
    import numpy as np
    o = []
    print np.array(a), np.array(a).shape
    s = np.array_split(np.array(a), w)
    return [np.median(a) for a in s]


In [ ]:
def convert_chrom_notation(chrom):
    """
    add 'chr' prefixes
    """
    if 'chr' in str(chrom):
        return str(chrom)
    else: return 'chr' + str(chrom)

## Read in the methylation and Red1 ChIP data

We'll need to make sure chromosome notation is constant in both files.

In [ ]:
# read in the Red1 data, and use new, simpler column names
red1 = pd.read_csv("red1-chip.tsv", sep="\t", header=0, 
                       names=["chrom", "pos", "background", "red1", "signal"])

# add chromosome prefixes
red1['chrom'] = red1['chrom'].apply(convert_chrom_notation)

# take a look at the data
red1.head()

In [ ]:
# read in the ONT methylation data
# rename "methylated_frequency" to "signal"
meth = pd.read_csv("methylation-frequency.tsv", sep='\t', header=0, 
                       names=["chrom", "pos", "end", "num_cpg_motifs", "depth", 
                              "methylated_reads", "signal", "sequence"])

meth.drop(columns=['end'], inplace=True)

# take a look at the data
meth.head()

## Plot the raw methylation and Red1 signals across the genome

This ends up being pretty messy!

In [ ]:
f, ax = plt.subplots(figsize=(15,5))

chrom = 'chr8'

meth_ = meth[meth['chrom'] == chrom]
red1_ = red1[red1['chrom'] == chrom]

ax2 = ax.twinx()

sns.scatterplot(x="pos", y="signal", data=red1_, ax=ax, color='firebrick')
sns.scatterplot(x="pos", y="signal", data=meth_, ax=ax2, color='dodgerblue')

## Bin genome-wide signals into windows 

After binning the existing data into windows, we'll calculate the mean signal per window. This makes the plots much easier to parse, but it might be too simplistic to simply take the per-window mean.

In [ ]:
from scipy.interpolate import BSpline
import scipy.stats as ss

# create the figure object
f, ax = plt.subplots(figsize=(15,5))
ax2 = ax.twinx()

# the chromosome we want to look at
chrom = 'chr8'

# subset the data to exclude sites with zero signal
# also, exclude methylation sites with fewer than 10 reads
meth_filt = meth.query('depth > 10 & signal > 0.25')
red1_filt = red1.query('signal > 10')

def bin_data(df, chrom='chr8', n_windows=100):
    """
    given a dataframe, subset to the chromosome of interest
    and calculate the median signal per window
    """
    # filter to only include the chromosome of interest
    df = df[df['chrom'] == chrom]
    # get arrays of positions and signals at those positions
    sites, signal = df['pos'].values, df['signal'].values
    # divide the data into `n_windows` and find the positions
    # that separate those windows (`edges`)
    _, edges = np.histogram(sites, n_windows)
    edges = edges[:-1]
    # calculate the mean signal per window
    mean_signal = mean_signal_per_window(signal, w=n_windows)
    
    return edges, mean_signal

# get the bin coordinates and mean signal per bin
meth_x, meth_y = bin_data(meth_filt, chrom=chrom, n_windows=100)
red1_x, red1_y = bin_data(red1_filt, chrom=chrom, n_windows=100)

ax.plot(red1_x, red1_y, color="dodgerblue", label="ChIP")
ax2.plot(meth_x, meth_y, color="firebrick", label="methylation")

ax.legend(loc="upper left")
ax.set_ylabel("Mean ChIP signal")
ax2.legend(loc="upper right")
ax2.set_ylabel("Mean methylation frequency")

## Smooth the genome-wide signals 

Here, we'll try interpolating the methylation and Red1 signals using a smaller set of x-values. In this way, we can smooth out the signal-vs-position plots a bit.

In [ ]:
from scipy.interpolate import BSpline
import scipy.stats as ss

# create the figure object
f, ax = plt.subplots(figsize=(15,5))
ax2 = ax.twinx()

# the chromosome we want to look at
chrom = 'chr16'
meth_ = meth[meth['chrom'] == chrom]
red1_ = red1[red1['chrom'] == chrom]

# get arrays of positions and signals for both methylation and Red1 ChIP data
meth_x, meth_y = meth_['pos'].values, meth_['signal'].values
red1_x, red1_y = red1_['pos'].values, red1_['signal'].values

# use interpolation to "smooth" the existing data
meth_x_new = np.linspace(min(meth_x), max(meth_x), 100)
meth_spl = BSpline(meth_x, meth_y, 3)
red1_x_new = np.linspace(min(red1_x), max(red1_x), 100)
red1_spl = BSpline(red1_x, red1_y, 3)

ax2 = ax.twinx()

# plot the interpolated points
sns.lineplot(meth_x_new, meth_spl(meth_x_new), color='firebrick', label='methylation', ax=ax)
sns.lineplot(red1_x_new, red1_spl(red1_x_new), color='dodgerblue', label='ChIP', ax=ax2)

ax.legend(loc="upper left")
ax.set_ylabel("Mean ChIP signal")
ax2.legend(loc="upper right")
ax2.set_ylabel("Mean methylation frequency")

## Plot regions of the genome with and without methylation or Red1 signal

Instead of plotting raw signals, let's try to convert every Red1 or methylation observation into a binary "yes/no" value, which simply represents the presence or absence of ChIP binding or methylation.

In [ ]:
def obs_per_window(a, w=1000):
    """
    count the number of events in
    non-overlapping windows of specified size
    
    a: list of integers
    """
    o = []
    bins = len(a) / w
    s, e = 0, w
    for b in range(bins):
        b_count = sum(a[s:e])
        o.append(b_count)
        s += w
        e += w
    return o

## Catalog every site where we observe Red1 binding signal

In [ ]:
# convert normalized Red1 signals to binary values
red1['binary_signal'] = red1['signal'].apply(to_binary)

chrom_sizes = defaultdict(int)

# get lists of binary Red1 signals (in 20 bp windows) for each chromosome
# also, record the max length of each chromosome
red1_signal = defaultdict(list)
for v in red1[['chrom', 'pos', 'binary_signal']].values:
    red1_signal[v[0]].append(v[2])
    if chrom_sizes[v[0]] < v[1]:
        chrom_sizes[v[0]] = v[1]

## Catalog every site where we observe methylation

Also, bin the observations into 20 bp bins, so that the data are equivalent to the Red1 data.

In [ ]:
# filter the data to include only high quality methylated sites
meth_filt = meth.query('depth > 10 & signal > 0')

# record the position of each high quality methylation event on each chromosome
methylation_counts = defaultdict(list)
for v in meth_filt[['chrom', 'pos']].values:
    methylation_counts[v[0]].append(v[1])
    
meth_signal = defaultdict(list)

# loop over every chromosome, and create a histogram of 
# methylation counts per 20 bp bin
for chrom in methylation_counts:
    if chrom == 'chrM': continue
    # create as many bins as needed to have one 
    # every 20 bp on the chromosome
    bins = int(chrom_sizes[chrom] / 20)
    hist, edges = np.histogram(methylation_counts[chrom], bins=bins)
    meth_signal[chrom] = list(hist)

## Plot the genome-wide distribution of methylation and Red1 counts

Here, we can bin the data further, since plotting thousands of 20 bp bins can be pretty messy.

In [ ]:
# create the figure object
f, ax = plt.subplots(figsize=(12,4))

chrom = 'chr6'
window = 200

# get the number of Red1 binding events or methylated sites
# per `window` * 20bp interval
m = obs_per_window(meth_signal[chrom], w=window)
r = obs_per_window(red1_signal[chrom], w=window)

# make sure the x values correspond to the true genomic positions
x = [i*20*window for i in range(len(m))]

ax2 = ax.twinx()

ax.plot(x, r, color='firebrick', label='Red1 signal')
ax2.plot(x, m, color='dodgerblue', label='Methylation signal')

ax.set_xlabel('Chromosome position')
ax.set_ylabel('Red1 binding counts per window')
ax2.set_ylabel('Methlated sites per window')

ax.legend()
ax2.legend()

## Use a hidden Markov model (HMM) to define broad regions of methylation or ChIP binding

This is super preliminary, but could be useful to segment the data into more usable chunks.

In [ ]:
from pomegranate import *

In [ ]:
red1_binding_sites = defaultdict(list)

# catalog every Red1 binding site as a binary "yes" or "no"
# binding event
for i,row in red1.iterrows():
    red1_binding_sites[row['chrom']].append(row['binary_signal'])
        
red1_hmm = defaultdict(list)

# loop over every chromosome and use an HMM to segment
# the binding profiles into broad regions
for idx, chrom in enumerate(red1_binding_sites):
    mx, my = [], []
    
    # define the expected profiles of "bound" or 
    # "unbound" regions (95% of sites should have Red1 signal
    # in a "bound" region, with up to 5% of sites without Red1 signal)
    d1 = DiscreteDistribution({1: 0.05, 0: 0.95})
    d2 = DiscreteDistribution({1: 0.95, 0: 0.05})

    s1 = State(d1, name='unbound')
    s2 = State(d2, name='bound')

    # inititate the model
    hmm = HiddenMarkovModel()
    hmm.add_states(s1, s2)
    hmm.add_transition(hmm.start, s1, 0.5)
    hmm.add_transition(hmm.start, s2, 0.5)
    # define the transition probabilities (i.e, the probability of
    # switching from a bound region to an unbound region). we intentionally
    # make these probabilities extreme so that the model doesn't randomly
    # switch back and forth if there's a little noise in the data
    hmm.add_transition(s1, s1, 0.99)
    hmm.add_transition(s1, s2, 0.01)
    hmm.add_transition(s2, s1, 0.01)
    hmm.add_transition(s2, s2, 0.99)
    hmm.bake()
    
    # use the HMM to predict the binding profiles on the chromosome
    hmm_predictions = hmm.predict(red1_binding_sites[chrom])
    
    switch_dict = {1:0, 0:1}
    
    if hmm.states[0].name == 'bound':
        hmm_predictions = [switch_dict[e] for e in hmm_predictions]
    
    # record this chromosome's binding profile
    red1_hmm[chrom] = hmm_predictions
print red1_hmm

## Plot reads that span multiple methylated sites

Also, quite preliminary, but a quick visualization.

In [ ]:
df = pd.read_csv("methylation.log-lik-pos.tsv", sep='\t')

In [ ]:
sites_per_read = defaultdict(list)

# catalog all of the sites with likely methylation events
# associated with a particular read
for i,row in df.iterrows():
    read_name = row['read_name']
    sites_per_read[read_name].append(row['chromosome'] + ':' + str(row['end']))

per_chrom_intervals = defaultdict(list)
    
# for every chromosome, store the full span of each read's methylation events
for read in sites_per_read:
    positions = [int(p.split(':')[-1]) for p in sites_per_read[read]]
    chroms = [p.split(':')[0] for p in sites_per_read[read]]
    d = defaultdict(list)
    for k,v in zip(chroms, positions):
        d[k].append(v)
    for chrom in d:
        per_chrom_intervals[chrom].append((min(d[chrom]), max(d[chrom]), read))

In [ ]:
f, axarr = plt.subplots(len(per_chrom_intervals), 1, figsize=(10, 60))

for idx,chrom in enumerate(per_chrom_intervals):
    for interval in per_chrom_intervals[chrom]:
        if interval[1] - interval[0] < 10000: continue
        if interval[1] - interval[0] > 100000: continue
            
        x = [interval[0], interval[1]]
        # add dither to reads when we plot them
        y = np.ones(2) + np.random.normal(loc=0, scale=1)
        
        methylated_sites = per_chrom_intervals[chrom]
        n = len(methylated_sites)
        for i,site in enumerate(methylated_sites):
            if i == n-1: break
            if site in range(x[0], x[1]) and methylated_sites[i+1] in range(x[0], x[1]):
                print 'read {} on chrom {} spans two methylated sites'.format(interval[-1],
                                                                              str(chrom) + ':' + str(x[0]) + '-' + str(x[1]))
        sites = [s[0] for s in methylated_sites]
        axarr[idx].plot(x, y, color='k')
        axarr[idx].scatter(sites, np.ones(len(sites)), 
                           facecolors='none', edgecolor='r')
        axarr[idx].set_title(chrom)
f.tight_layout()